In [17]:
%matplotlib inline
import os
#os.chdir('U:\\GitHubClones\\SeniorDesign')
#os.getcwd()
#import Excel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime

In [18]:
os.chdir('C:\\Users\\austi\\Desktop\\Senior Design Project\\Files')

In [19]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [22]:
#path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.dealName + "\\" + Excel.reportList[2]
june = '2017.06.15_Santander Drive Auto Receivables Trust 2017-1 Data Tape.csv'
july = '2017.07.17_Santander Drive Auto Receivables Trust 2017-1 Data Tape.csv'
path = 'C:\\Users\\austi\\Desktop\\Senior Design Project\\Files' + '\\' + june
print(path)

C:\Users\austi\Desktop\Senior Design Project\Files\2017.06.15_Santander Drive Auto Receivables Trust 2017-1 Data Tape.csv


In [23]:
df = pd.read_csv(path, memory_map=True, low_memory=False)
len(df.index)


64551

In [39]:
list(df.columns)

['assetTypeNumber',
 'assetNumber',
 'reportingPeriodBeginningDate',
 'reportingPeriodEndingDate',
 'originatorName',
 'originationDate',
 'originalLoanAmount',
 'originalLoanTerm',
 'loanMaturityDate',
 'originalInterestRatePercentage',
 'interestCalculationTypeCode',
 'originalInterestRateTypeCode',
 'originalInterestOnlyTermNumber',
 'originalFirstPaymentDate',
 'underwritingIndicator',
 'gracePeriodNumber',
 'paymentTypeCode',
 'subvented',
 'vehicleManufacturerName',
 'vehicleModelName',
 'vehicleNewUsedCode',
 'vehicleModelYear',
 'vehicleTypeCode',
 'vehicleValueAmount',
 'vehicleValueSourceCode',
 'obligorCreditScoreType',
 'obligorCreditScore',
 'obligorIncomeVerificationLevelCode',
 'obligorEmploymentVerificationCode',
 'coObligorIndicator',
 'paymentToIncomePercentage',
 'obligorGeographicLocation',
 'assetAddedIndicator',
 'remainingTermToMaturityNumber',
 'reportingPeriodModificationIndicator',
 'servicingAdvanceMethodCode',
 'reportingPeriodBeginningLoanBalanceAmount',
 '

In [6]:
df['obligorIncomeVerificationLevelCode'].unique()

array([3, 2], dtype=int64)

In [7]:
WA = lambda x: np.average(x, axis=0, weights = df.loc[x.index, 'reportingPeriodActualEndBalanceAmount']) #have to calculate weighted average values before performing loan Cut

In [26]:
df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio
df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100 #Individual Loan as a Percent of Current month Balance
df['totalCollected'] = df['actualInterestCollectedAmount'] + df['actualPrincipalCollectedAmount'] + df['actualOtherCollectedAmount']
df['principalPaid'] = df['originalLoanAmount'] - df['reportingPeriodActualEndBalanceAmount']


In [ ]:
preDefault = df[['assetNumber','originalLoanAmount', 'reportingPeriodActualEndBalanceAmount', 'principalPaid','totalActualAmountPaid','totalCollected']]

print(df['pctDeal'].head())
preDefault.head()

In [17]:
for i in range(0,10):
    path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.dealName + "\\" + Excel.reportList[i]
    df = pd.read_csv(path, memory_map=True, low_memory=False)
    
    df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount'])*100 #Loan to value ratio
    df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100
    totalBalAmount = df['reportingPeriodActualEndBalanceAmount'].sum()
    
    
    df = df[~df.zeroBalanceCode.isin(['-'])] #taking out all of the performing loans so far
    df['zeroBalanceCode'] = pd.to_numeric(df['zeroBalanceCode'], errors='coerce')
    df.loc[df['zeroBalanceCode'] > 1] = 0
    

'\nfor i in range(0,10):\n    path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.dealName + "\\" + Excel.reportList[i]\n    df = pd.read_csv(path, memory_map=True, low_memory=False)\n\n'

### Converting Zero Balance Codes

In [111]:
df = df[~df.zeroBalanceCode.isin(['-'])]
print(len(df.index))
print(df['zeroBalanceCode'].unique())
df['zeroBalanceCode'] = pd.to_numeric(df['zeroBalanceCode'], errors='coerce')
print(df['zeroBalanceCode'].dtype)
df['zeroBalanceCode'].head()

1769
['4' '1' '3']
int64


4      4
78     4
100    4
151    1
155    4
Name: zeroBalanceCode, dtype: int64

In [112]:
df['recoveredAmount'].unique()#
df['chargedoffPrincipalAmount'].unique()
df['repossessedProceedsAmount'].unique()#object
df['subvented'].unique()

array(['0', '1', '1;  2'], dtype=object)

In [113]:
df.loc[df['zeroBalanceCode'] > 1] = 0 # 1 means the loan has paid down, 0 means the loan has defaulted in some way
df['zeroBalanceCode'].unique()
df['zeroBalanceCode'].head()

4      0
78     0
100    0
151    1
155    0
Name: zeroBalanceCode, dtype: int64

#### Changing Values to Numeric

In [117]:
df['obligorCreditScore'] = pd.to_numeric(df['obligorCreditScore'], errors='coerce')
df['subvented'] = pd.to_numeric(df['subvented'], errors='coerce')
df['obligorIncomeVerificationLevelCode'].unique()

array([0, 2, 3], dtype=int64)

### Bins For Cutting Data

In [115]:
FicoBins = [-1, 440, 500, 560, 620, 680, 740, 800, 1000]
APRBins = [0, .02 ,.04, .06, .08, .1, .12, .14, .16, 1000]
LTVBins = [-1, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1000]
PTIBins = [-1, .02, .04, .06, .08, .1, .12, .14, .16, 1000]
ZeroBalanceCodeBins = [-1, 0, 1]
SubventedCodeBins = [-1, 0, 1, 2, 3]
IncomeVerificationCodeBins = [0, 1, 2, 3, 4, 5]

In [116]:
FicoGroup = ['[-1, 440)', '[440, 500)', '[500, 560)', '[560, 620)', '[620, 680)', '[680, 740)', '[740, 800)', '[800, 1000)']
APRGroup = ['[0.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
PTIGroup = ['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
LTVGroup = ['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)', '[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000.0)']
ZeroBalanceCodeGroup = ['Default', 'Paid Down']
SubventedCodeGroup = ['None', 'Rate', 'Cash', 'Other']
IncomeVerificationCodeGroup = ['None', 'Stated', 'Stated & Verified', 'Extra Verfied', '2 Years Verified']

### Cutting the Data

In [106]:
df['Fico Group'] = pd.cut(df['obligorCreditScore'], FicoBins, right=True, labels=FicoGroup)
df['APR Group'] = pd.cut(df['originalInterestRatePercentage'], APRBins, right=True, labels=APRGroup)
df['PTI Group'] = pd.cut(df['paymentToIncomePercentage'], PTIBins, right=True, labels=PTIGroup)
df['Zero Balance Code Group'] = pd.cut(df['zeroBalanceCode'], ZeroBalanceCodeBins, right=True, labels=ZeroBalanceCodeGroup)
df['Subvented Code Group'] = pd.cut(df['subvented'], SubventedCodeBins, right=True, labels=SubventedCodeGroup)
df['Income Verification Code Group'] = pd.cut(df['obligorIncomeVerificationLevelCode'], IncomeVerificationCodeBins, right=True, labels=IncomeVerificationCodeGroup)
df['Income Verification Code Group'].head()

4         NaN
78        NaN
100       NaN
151    Stated
155       NaN
Name: Income Verification Code Group, dtype: category
Categories (5, object): [2 Years Verified < Extra Verfied < None < Stated < Stated & Verified]

#### Adding No Score Category

In [85]:
df['Fico Group'] = df['Fico Group'].cat.add_categories('No Score').fillna('No Score')

In [86]:
a = OrderedDict([
    ('assetNumber', ['count']),
    ('originalInterestRatePercentage', ['mean']),
    ('chargedoffPrincipalAmount', ['mean']),
    ('totalActualAmountPaid', ['mean'])
])

In [87]:
df['Zero Balance Code Group'].value_counts()

Default      943
Paid Down    826
Name: Zero Balance Code Group, dtype: int64

In [89]:
df[['assetNumber', 'originalInterestRatePercentage', 'chargedoffPrincipalAmount', 'totalActualAmountPaid']].groupby([ df['Fico Group'], df['Subvented Code Group'], df['Zero Balance Code Group']]).agg(a)

assetNumber  \
                                                               count   
Fico Group  Subvented Code Group Zero Balance Code Group               
[-1, 440)   None                 Default                         943   
                                 Paid Down                         1   
[440, 500)  None                 Paid Down                        20   
            Rate                 Paid Down                         1   
[500, 560)  None                 Paid Down                       110   
            Rate                 Paid Down                        14   
[560, 620)  None                 Paid Down                       228   
            Rate                 Paid Down                        35   
[620, 680)  None                 Paid Down                       190   
            Rate                 Paid Down                        14   
[680, 740)  None                 Paid Down                        64   
            Rate                 Paid Down                         1   
[740, 800)  None                 Paid Down                        12   
[800, 1000) None                 Paid Down                        10   
No Score    None                 Paid Down                       114   
            Rate                 Paid Down                        12   

                                                         originalInterestRatePercentage  \
                                                                                   mean   
Fico Group  Subvented Code Group Zero Balance Code Group                                  
[-1, 440)   None                 Default                                       0.000000   
                                 Paid Down                                     0.240000   
[440, 500)  None                 Paid Down                                     0.202220   
            Rate                 Paid Down                                     0.180000   
[500, 560)  None                 Paid Down                                     0.182687   
            Rate                 Paid Down                                     0.144129   
[560, 620)  None                 Paid Down                                     0.163616   
            Rate                 Paid Down                                     0.121583   
[620, 680)  None                 Paid Down                                     0.149204   
            Rate                 Paid Down                                     0.129614   
[680, 740)  None                 Paid Down                                     0.136195   
            Rate                 Paid Down                                     0.044300   
[740, 800)  None                 Paid Down                                     0.116083   
[800, 1000) None                 Paid Down                                     0.074800   
No Score    None                 Paid Down                                     0.205929   
            Rate                 Paid Down                                     0.137458   

                                                         chargedoffPrincipalAmount  \
                                                                              mean   
Fico Group  Subvented Code Group Zero Balance Code Group                             
[-1, 440)   None                 Default                                       0.0   
                                 Paid Down                                     0.0   
[440, 500)  None                 Paid Down                                     0.0   
            Rate                 Paid Down                                     0.0   
[500, 560)  None                 Paid Down                                     0.0   
            Rate                 Paid Down                                     0.0   
[560, 620)  None                 Paid Down                                     0.0   
            Rate                 Paid Down                                     0.0   
[620, 680)  